<a href="https://colab.research.google.com/github/SteffiJF/wine-specialization-project/blob/main/MachineLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Constructing features and targets and running LSTM models 

In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from google.colab import files
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import drive
from oauth2client.client import GoogleCredentials	
from datetime import datetime
%matplotlib inline
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from datetime import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
import keras.backend as K
from matplotlib import pyplot
from numpy import array
from numpy import hstack
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE, ADASYN
import seaborn as sns
import matplotlib._color_data as mcd
# Use seaborn style defaults and set the default figure size
sns.set(rc={'figure.figsize':(10, 6)})


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


Loading data from Google Disk

In [3]:
#Settting up connection to Google Disk
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#Loading district sales data
download = drive.CreateFile({'id': '1TQiphmTgnwQCpPsuOJDEPL59UwXU18Kv'})
download.GetContentFile('aggData')
df = pd.read_csv("aggData")

#Loading wine sales data
download = drive.CreateFile({'id': '1_EI5mruMITjXloABXYyo5BGMo-mEzpj1'})
download.GetContentFile('topWines')
dfW = pd.read_csv("topWines")

#Loading baseline analysis
download = drive.CreateFile({'id': '14OvekNaapSeGIt8HTM9KqCj7XYqjpShz'})
download.GetContentFile('SarimaY')
dfSY = pd.read_csv("SarimaY")

download = drive.CreateFile({'id': '1pW-eKW_zNqPVB82CCKufDWro8cRXbNof'}) 
download.GetContentFile('SarimaY2')
dfSY2 = pd.read_csv("SarimaY2")

download = drive.CreateFile({'id': '1f-3XyHPP-gtlRFHHkTENcKl_If8C7Mt2'}) 
download.GetContentFile('SarimaY3')
dfSY3 = pd.read_csv("SarimaY3")

#Removing old index
dfW = dfW.drop('Unnamed: 0', axis=1)
dfSY = dfSY.drop('Unnamed: 0', axis=1)
dfSY2 = dfSY2.drop('Unnamed: 0', axis=1)
dfSY3 = dfSY3.drop('Unnamed: 0', axis=1)

#Setting index to yyyy-mm-dd
df['År_Måned'] = pd.to_datetime(df['År_Måned'])
df = df.set_index('År_Måned')
dfW['År_Måned'] = pd.to_datetime(dfW['År_Måned'])
dfW = dfW.set_index(['Artikkelnr','År_Måned']).sort_index()

#Removing areas that did not pass visual inspection of plots

#List of areas
areas = df.columns
#print(areas)
#Returns string with area name from index
def area_name(i):
  return areas[i]

#Makes a list of area names that will be removed
def areas_to_remove(removeUncertainAreas = True):
  badAreaIndex = [0,1,3,9,10,14,19,25,28,29,37,40,53,64,68,69]#[0,3,9,10,14,19,25,28,29,37,40,53,64,69]
  uncertainAreaIndex = [1,11,15,17,22,30,34,43,44,66,71]#fjernet 68 og la til over
  badAreas = []
  for i in badAreaIndex:
    badAreas.append(area_name(i))
  if removeUncertainAreas == True:
    for i in uncertainAreaIndex:
      badAreas.append(area_name(i))
  return badAreas

badAreas = areas_to_remove(removeUncertainAreas=False)

for area in badAreas:
  df.drop(area, axis=1, inplace=True)

df.to_csv('final_agg_data.csv',sep=';')

#Updated list of areas
areas = df.columns
#print(len(areas))

#print(areas)
val_areas = areas

#Updating columns names
dfSY.columns = areas
dfSY2.columns = areas
dfSY3.columns = areas

#Updating index names
dfSY.index = df.index
dfSY2.index = df.index
dfSY3.index = df.index

Constructing different features for aggregated data

In [4]:
#Features for X-values

#Differencing with lag 12
dfDiff = df.diff(periods=12)

#Percentwice differencing with lag 12
dfPctDiff = df.pct_change(periods=12)

#Soft clamping dfPctDiff to avoid large peaks
dfSoftPctDiff = np.arcsinh(dfPctDiff)

#Dividing differenced data by shifted rolling standard deviation 
dfStdDiff = dfDiff/dfDiff.shift(1).rolling(12).std()
dfStdPctDiff = dfPctDiff/dfPctDiff.shift(1).rolling(12).std()
dfStdSoftPctDiff = dfSoftPctDiff/dfSoftPctDiff.shift(1).rolling(12).std()

#Finding positive and negative spikes 
#dfSpikes = dfStdDiff.apply(lambda x: [1 if y >= 3 else (-1 if y<=-3 else 0) for y in x])
dfSpikes = dfStdPctDiff.apply(lambda x: [1 if y >= 3 else (-1 if y<=-3 else 0) for y in x])
#dfSmallSpikes = dfStdDiff.apply(lambda x: [1 if y >= 2 else (-1 if y<=-2 else 0) for y in x])


#Finding the rolling mean of spikes (testing width 3, can reconsider width later).
#These are not shifted, meaning that the sum for a row is the sum of that row and the two previous rows
dfSpikes = dfSpikes.rolling(3).sum()

#Creating categories 1,0, and -1 depending on the majority of spikes
dfSpikes = dfSpikes.apply(lambda x: [1 if y > 0 else (-1 if y < 0 else 0) for y in x])

#Replacing inf with 0 where necessary
df = df.replace([np.inf, -np.inf], 0)
dfDiff = dfDiff.replace([np.inf, -np.inf], 0)
dfPctDiff = dfPctDiff.replace([np.inf, -np.inf], 0)
dfSoftPctDiff = dfSoftPctDiff.replace([np.inf, -np.inf], 0)
dfStdDiff = dfStdDiff.replace([np.inf, -np.inf], 0)
dfStdPctDiff = dfStdPctDiff.replace([np.inf, -np.inf], 0)
dfStdSoftPctDiff = dfStdPctDiff.replace([np.inf, -np.inf], 0)

#Removing nan values
df = df.fillna(0)
dfDiff = dfDiff.fillna(0)
dfPctDiff = dfPctDiff.fillna(0)
dfSoftPctDiff = dfSoftPctDiff.fillna(0)
dfStdDiff = dfStdDiff.fillna(0)
dfStdPctDiff = dfStdPctDiff.fillna(0)
dfStdSoftPctDiff = dfStdSoftPctDiff.fillna(0)


#Setting up Y-values


#First classification, checks percentage-wise difference of coming year's sum vs previous year's sum
dfY = df.rolling(12).sum().shift(-12)
dfY = dfY.pct_change(periods=12)
dfY= dfY.apply(lambda x: [2 if y >= 0.3 else (0 if y<=-0.3 else 1) for y in x])

#Second calssification, checks the mean of the coming years difference divided by standard deviation
dfY2 = dfStdDiff.rolling(12).mean().shift(-12)
dfY2 = dfY2.apply(lambda x: [2 if y >= 1 else (0 if y<=-1 else 1) for y in x])

#Third classification, checks the amount of small positive spikes in the percentage-wise change divided by standard deviation
dfSmallSpikes2 = dfStdPctDiff.apply(lambda x: [1 if y >= 2 else (-1 if y<=-2 else 0) for y in x])
dfY3 = dfSmallSpikes2.rolling(12).sum().shift(-12)
dfY3 = dfY3.apply(lambda x: [2 if y >= 3 else (0 if y<=-3 else 1) for y in x])


#Removing nan values, set to 1 which is neutral
dfY = dfY.fillna(1)
dfY2 = dfY2.fillna(1)
dfY3 = dfY3.fillna(1)


Top 5 wines for each district

In [5]:
#Extracting article numbers from dataframe
articles = dfW.index.values.tolist()
articles = [row[0] for row in articles]
articles = list(dict.fromkeys(articles))


#Finding an arbitrary article with 154 rows and adding to empty dataframe
def set_df_length(df, oldArticles):
  for art in articles:
    if len(dfW.loc[art].index.values)==154:
      df[art] = dfW.loc[art]['Liter']
      oldArticles = oldArticles + [art]
      return df, oldArticles

#List for storing articles already added to dataframe 
oldArticles = []

#Dataframe for storing all of the liters sold with article nr as columne name
dfWL = pd.DataFrame()

#Adding a first column to the dataframe to assure correct length
dfWL, oldArticles = set_df_length(dfWL, oldArticles)

#Dataframe for storing which articles come from which area
dfAA = pd.DataFrame()

#Looping through all of the areas and adding data to dfWL and dfAA
for area in areas:
  dfTemp=dfW[dfW['Område']==area]
  if not dfTemp.empty:
    articles = dfTemp.index.values.tolist()
    articles = [row[0] for row in articles]
    articles = list(dict.fromkeys(articles))
    if len(articles)==5:
      dfAA[area] = articles
      #print(articles)
      for i in range(0,5):
        #Avoiding adding duplicated wines twice
        if articles[i] not in oldArticles:
          #print(articles[i])
          dfWL[articles[i]] = dfTemp.loc[articles[i]]['Liter']
      oldArticles = oldArticles + articles
      #print(oldArticles)



#Replacing NaN with 0.
dfWL = dfWL.fillna(0)


x = dfWL.values #returns a numpy array
min_max_scaler = MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
dfWL = pd.DataFrame(x_scaled, columns= dfWL.columns)




Calculating results of ARIMA baseline

In [6]:
#Initializing lists and fnuctions
listY=[]
listSY=[]
listY2=[]
listSY2=[]
listY3=[]
listSY3=[]
precisionS=np.zeros(3)
precisionS2=np.zeros(3) 
precisionS3=np.zeros(3)

def precision(label, confusion_matrix):
    col = confusion_matrix[:, label]
    return confusion_matrix[label, label] / col.sum()


#Gathering test values for all districts in lists
for i in range(len(areas)):
  listY = np.append(listY, dfY[areas[i]][122:141])
  listSY = np.append(listSY, dfSY[areas[i]][122:141])
  listY2 = np.append(listY2, dfY2[areas[i]][122:141])
  listSY2 = np.append(listSY2, dfSY2[areas[i]][122:141])
  listY3 = np.append(listY3, dfY3[areas[i]][122:141])
  listSY3 = np.append(listSY3, dfSY3[areas[i]][122:141])

#Comparing ARIMA results to true values

#Y
precisionsS=precision_recall_fscore_support(listY, listSY, beta=1.0, labels=None, average='macro')[0]
recallS=precision_recall_fscore_support(listY, listSY, beta=1.0, labels=None, average='macro')[1]
f1scoreS=precision_recall_fscore_support(listY, listSY, beta=1.0, labels=None, average='macro')[2]
micro_f1scoreS = precision_recall_fscore_support(listY, listSY, beta=1.0, labels=None, average='micro')[2]


#Y2
precisionsS2=precision_recall_fscore_support(listY2, listSY2, beta=1.0, labels=None, average='macro')[0]
recallS2=precision_recall_fscore_support(listY2, listSY2, beta=1.0, labels=None, average='macro')[1]
f1scoreS2=precision_recall_fscore_support(listY2, listSY2, beta=1.0, labels=None, average='macro')[2]
micro_f1scoreS2 = precision_recall_fscore_support(listY2, listSY2, beta=1.0, labels=None, average='micro')[2]


#Y3
precisionsS3=precision_recall_fscore_support(listY3, listSY3, beta=1.0, labels=None, average='macro')[0]
recallS3=precision_recall_fscore_support(listY3, listSY3, beta=1.0, labels=None, average='macro')[1]
f1scoreS3=precision_recall_fscore_support(listY3, listSY3, beta=1.0, labels=None, average='macro')[2]
micro_f1scoreS3 = precision_recall_fscore_support(listY3, listSY3, beta=1.0, labels=None, average='micro')[2]


#Closer precision analysis
cmS = confusion_matrix(listY, listSY, labels=[0, 1, 2])
cmS2 = confusion_matrix(listY2, listSY2, labels=[0, 1, 2])
cmS3 = confusion_matrix(listY3, listSY3, labels=[0, 1, 2])

for i in range(0,3):
  precisionS[i]=precision(i, cmS)
  precisionS2[i]=precision(i, cmS2)  
  precisionS3[i]=precision(i, cmS3)

#Printing results
print('Y')
print(precisionsS,recallS,f1scoreS,micro_f1scoreS)
print(precisionS)

print('Y2')
print(precisionsS2,recallS2,f1scoreS2,micro_f1scoreS2)
print(precisionS2)

print('Y3')
print(precisionsS3,recallS3,f1scoreS3,micro_f1scoreS3)
print(precisionS3)


Y
0.49873677111799103 0.4210257428071568 0.39155414389251847 0.6889097744360902
[0.42857143 0.75220264 0.31543624]
Y2
0.3034410667425464 0.38664834076356475 0.3250116509251346 0.49530075187969924
[0.07692308 0.57090012 0.2625    ]
Y3
0.4716619981325863 0.3500527092557453 0.29088556396922416 0.5366541353383458
[0.7        0.57773109 0.1372549 ]


Preparing data for machine learning

In [10]:
#Constructing train and test data

n_steps = 15

nr_sets = len(df)-26-12-(n_steps-1) #26 for beginning, 12 at end, 14 for extra months
nr_test_months = round(nr_sets*0.2)
nr_train_months = round((nr_sets-nr_test_months)*0.8)


start_train_months = 26 
end_train_months= start_train_months+nr_train_months+ n_steps-1
end_test_months = len(df)-13 #1 for index and 12 for months without y-values 
start_test_months = end_test_months - nr_test_months-n_steps+2 

start_val_months = end_train_months-n_steps+2+6 #+6 for quarantine period between train and validation
end_val_months = start_test_months + n_steps-1

areas = dfAA.columns  

#Scaling data

def scale(df):
  x = df.values 
  min_max_scaler = MinMaxScaler()
  x_scaled = min_max_scaler.fit_transform(x)
  #robust_scaler = RobustScaler()
  #x_scaled = robust_scaler.fit_transform(x)
  return pd.DataFrame(x_scaled, columns= df.columns)



dfS = scale(df)
dfDiffS = scale(dfDiff)
dfPctDiffS = scale(dfPctDiff)
dfSoftPctDiffS = scale(dfSoftPctDiff)
dfStdDiffS = scale(dfStdDiff)
dfStdPctDiffS = scale(dfStdPctDiff)
dfStdSoftPctDiffS = scale(dfStdSoftPctDiff)
dfSpikesS = scale(dfSpikes)
dfWLS = scale(dfWL) 


#Function from Jason Brownlee
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


#Shapes features correctly for specified months and splits the sequences 
def shape_data(i,start_months, end_months):

  x1=array(dfS[areas[i]][start_months:end_months])
  x2=array(dfDiffS[areas[i]][start_months:end_months])
  x3=array(dfPctDiffS[areas[i]][start_months:end_months])
  x4=array(dfSoftPctDiffS[areas[i]][start_months:end_months])
  x5=array(dfStdDiffS[areas[i]][start_months:end_months])
  x6=array(dfStdPctDiffS[areas[i]][start_months:end_months])
  x7=array(dfStdSoftPctDiffS[areas[i]][start_months:end_months])
  x8=array(dfSpikesS[areas[i]][start_months:end_months])
  
  #Top wine sales
  x9=array(dfWLS[dfAA[areas[i]][0]][start_months:end_months])
  x10=array(dfWLS[dfAA[areas[i]][1]][start_months:end_months])
  x11=array(dfWLS[dfAA[areas[i]][2]][start_months:end_months])
  x12=array(dfWLS[dfAA[areas[i]][3]][start_months:end_months])
  x13=array(dfWLS[dfAA[areas[i]][4]][start_months:end_months])
  
  y=array(dfY[areas[i]][start_months:end_months])

  x1=x1.reshape((len(x1),1))
  x2=x2.reshape((len(x2),1))
  x3=x3.reshape((len(x3),1))
  x4=x4.reshape((len(x4),1))
  x5=x5.reshape((len(x5),1))
  x6=x6.reshape((len(x6),1))
  x7=x7.reshape((len(x7),1))
  x8=x8.reshape((len(x8),1))
  x9=x9.reshape((len(x9),1))
  x10=x10.reshape((len(x10),1))
  x11=x11.reshape((len(x11),1))
  x12=x12.reshape((len(x12),1))
  x13=x13.reshape((len(x13),1))
  y=y.reshape((len(y),1))

  #Change this to decide which features to use
  dataset = hstack(( x1, x2, x3, x4, x5, x6, x7, x9, y)) 

  return split_sequences(dataset, n_steps)


#Making the training data

X,y = shape_data(0,start_train_months,end_train_months)

for i in range(1,len(areas)):
  Xnew, ynew = shape_data(i,start_train_months,end_train_months)
  #Appending sequences together
  X = np.vstack((X,Xnew))
  y = np.append(y,ynew)


#Making the validation data

Xval,yval = shape_data(0,start_val_months,end_val_months)

for i in range(1,len(areas)):
  Xnew, ynew = shape_data(i,start_val_months,end_val_months)
  #Appending sequences together
  Xval = np.vstack((Xval,Xnew))
  yval = np.append(yval,ynew)


#Making the test data

Xtest,ytest = shape_data(0,start_test_months,end_test_months)

for i in range(1,len(areas)):
  Xnew, ynew = shape_data(i,start_test_months,end_test_months)
  #print(i,len(ynew))
  #Appending sequences together
  Xtest = np.vstack((Xtest,Xnew))
  ytest = np.append(ytest,ynew)

print('Before resampling')
print(sorted(Counter(y).items()))
print(sorted(Counter(yval).items()))
print(sorted(Counter(ytest).items()))


#Balancing the dataset by using random sampler on indices instead of X, as X 
#has 3D instead of 2D which is the limit

#Train

indices = np.arange(y.shape[0]).reshape(-1,1)

ros = RandomOverSampler(random_state=0)
indexResampled, y = ros.fit_resample(indices, y)

#Loops through the repeated indices, as these are placed after the original values
for i in range(len(indices), len(indexResampled)):
  #Appends 2D arrays from X to X by identifying them with index from indexResampled
  #The X[sample] is reshaped to have the same shape as X
  X = np.append(X,X[indexResampled[i]].reshape(1,X.shape[1],X.shape[2]), axis=0)


def resampler(X,y):
  indices = np.arange(y.shape[0]).reshape(-1,1)

  ros = RandomOverSampler(random_state=0)
  indexResampled, y = ros.fit_resample(indices, y)

  #Loops through the repeated indices, as these are placed after the original values
  for i in range(len(indices), len(indexResampled)):
    #Appends 2D arrays from X to X by identifying them with index from indexResampled
    #The X[sample] is reshaped to have the same shape as X
    X = np.append(X,X[indexResampled[i]].reshape(1,X.shape[1],X.shape[2]), axis=0)
  
  return X,y

X,y = resampler(X,y)


#Shuffling the input the same way for X and y

#New list of indices with additional samples
indices = np.arange(y.shape[0])
np.random.seed(0)
np.random.shuffle(indices)

X = X[indices]
y = y[indices]

y= y.astype(int)
yval = yval.astype(int)
ytest = ytest.astype(int)


print('After resampling')
print(sorted(Counter(y).items()))
print(sorted(Counter(yval).items()))
print(sorted(Counter(ytest).items()))


Before resampling
[(0.0, 590), (1.0, 2573), (2.0, 533)]
[(0.0, 55), (1.0, 367), (2.0, 82)]
[(0.0, 189), (1.0, 711), (2.0, 164)]


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


After resampling
[(0, 2573), (1, 2573), (2, 2573)]
[(0, 55), (1, 367), (2, 82)]
[(0, 189), (1, 711), (2, 164)]


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


LSTM model

In [11]:
#Importing learning rate finder, used for finding optimal learning rate
#https://github.com/surmenok/keras_lr_finder

#src = list(files.upload().values())[0]
#open('lr_finder.py','wb').write(src)
#import lr_finder
#from lr_finder import LRFinder


#Multivariate LSTM model for categorical prediction

n_features = X.shape[2]
n_outputs = 3
runs = 10 

#Preparation for analysis
accuracies = np.zeros(runs)
precisions = np.zeros(runs)
recalls = np.zeros(runs)
f1scores = np.zeros(runs)
micro_f1scores = np.zeros(runs)
precision0 = np.zeros(runs)
precision1 = np.zeros(runs)
precision2 = np.zeros(runs)


#Code from https://www.python-course.eu/confusion_matrix.php

def precision(label, confusion_matrix):
    col = confusion_matrix[:, label]
    return confusion_matrix[label, label] / col.sum()
    
def recall(label, confusion_matrix):
    row = confusion_matrix[label, :]
    return confusion_matrix[label, label] / row.sum()

def precision_macro_average(confusion_matrix):
    rows, columns = confusion_matrix.shape
    sum_of_precisions = 0
    for label in range(rows):
        sum_of_precisions += precision(label, confusion_matrix)
    return sum_of_precisions / rows

def recall_macro_average(confusion_matrix):
    rows, columns = confusion_matrix.shape
    sum_of_recalls = 0
    for label in range(columns):
        sum_of_recalls += recall(label, confusion_matrix)
    return sum_of_recalls / columns


#Defining model

for i in range(runs):
  model = Sequential()
  model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features))) 
  model.add(Dense(n_outputs))
  optimizer = tf.keras.optimizers.Adam(lr=0.0001) #Default lr is 0.001
  model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])



  #Used to find optimal learning rate
  #lr_finder = LRFinder(model)
  #lr_finder.find(X, y, 0.00001, 1, 32, 23)
  #lr_finder.plot_loss()


  #Fitting model
  history = model.fit(X, y, validation_data =(Xval, yval), batch_size=32, epochs=20, verbose=1)

  #Used to find optimal epochs
  #from keras import callbacks 
  #earlystopping = callbacks.EarlyStopping(monitor ="val_loss",  
  #                                        mode ="min", patience = 5,  
  #                                        restore_best_weights = True) 
    
  #history = model.fit(X, y,   
  #                    epochs = 100, batch_size=32, validation_data = (Xval, yval),  
  #                    callbacks =[earlystopping]) #fjernet batch size



  #Evaluating model
  _, accuracy = model.evaluate(Xtest, ytest, verbose=0)
  print(i)

  prediction = np.argmax(model.predict(Xtest), axis=-1)
  cm = confusion_matrix(ytest, prediction, labels=[0, 1, 2])

  label=[0,1,2]
  print("label precision recall")
  for label in range(3):
    print(f"{label:5d} {precision(label, cm):9.3f} {recall(label, cm):6.3f}")

  print('Accuracy: %.2f' % (accuracy*100))
  print("precision total:", precision_macro_average(cm))
  print("recall total:", recall_macro_average(cm))

  
  accuracies[i]=accuracy
  precisions[i]=precision_recall_fscore_support(ytest, prediction, beta=1.0, labels=None, average='macro')[0]
  recalls[i]=precision_recall_fscore_support(ytest, prediction, beta=1.0, labels=None, average='macro')[1]
  f1scores[i]=precision_recall_fscore_support(ytest, prediction, beta=1.0, labels=None, average='macro')[2]
  micro_f1scores[i] = precision_recall_fscore_support(ytest, prediction, beta=1.0, labels=None, average='micro')[2]

  precision0[i]=precision(0, cm)
  precision1[i]=precision(1, cm)  
  precision2[i]=precision(2, cm)

print('')
print('Accuracies:', np.average(accuracies))
#print(accuracies)

print('Precisions:', np.average(precisions))
#print(precisions)

print('Recalls:', np.average(recalls))
#print(recalls)

print('F1-scores:', np.average(f1scores), 'std:', np.std(f1scores))

print('F1-scores micro:', np.average(micro_f1scores), 'std:', np.std(micro_f1scores))

print('Precision class 0:', np.average(precision0))
#print(precision0)

print('Precision class 1:', np.average(precision1))
#print(precision1)

print('Precision class 2:', np.average(precision2))
#print(precision2)



Epoch 1/20
242/242 [==============================] - 3s 9ms/step - loss: 1.0787 - accuracy: 0.4313 - val_loss: 0.9574 - val_accuracy: 0.6131
Epoch 2/20
242/242 [==============================] - 2s 7ms/step - loss: 1.0076 - accuracy: 0.4978 - val_loss: 0.9327 - val_accuracy: 0.5694
Epoch 3/20
242/242 [==============================] - 2s 7ms/step - loss: 0.9186 - accuracy: 0.5925 - val_loss: 0.9467 - val_accuracy: 0.6329
Epoch 4/20
242/242 [==============================] - 2s 7ms/step - loss: 0.8871 - accuracy: 0.6133 - val_loss: 0.8368 - val_accuracy: 0.6687
Epoch 5/20
242/242 [==============================] - 2s 7ms/step - loss: 0.8575 - accuracy: 0.6295 - val_loss: 0.8874 - val_accuracy: 0.6508
Epoch 6/20
242/242 [==============================] - 2s 7ms/step - loss: 0.8283 - accuracy: 0.6445 - val_loss: 0.8356 - val_accuracy: 0.6687
Epoch 7/20
242/242 [==============================] - 2s 7ms/step - loss: 0.7985 - accuracy: 0.6609 - val_loss: 0.8520 - val_accuracy: 0.6567
Epoch 

Majority class classifier baseline

In [12]:
one_pred_dfY=np.ones(len(prediction))

cm = confusion_matrix(ytest, one_pred_dfY, labels=[0, 1, 2])

print(sorted(Counter(one_pred_dfY).items()))
print(sorted(Counter(ytest).items()))

label=[0,1,2]
print("label precision recall")
for label in range(3):
  print(f"{label:5d} {precision(label, cm):9.3f} {recall(label, cm):6.3f}")

print('Accuracy: %.2f' % (accuracy*100))
print("precision total:", precision_macro_average(cm))
print("recall total:", recall_macro_average(cm))

accuracies[i]=accuracy
precisions[i]=precision_macro_average(cm)
recalls[i]=recall_macro_average(cm)

precision0[i]=precision(0, cm)
precision1[i]=precision(1, cm)  
precision2[i]=precision(2, cm)

[(1.0, 1064)]
[(0, 189), (1, 711), (2, 164)]
label precision recall
    0       nan  0.000
    1     0.668  1.000
    2       nan  0.000
Accuracy: 72.37
precision total: nan
recall total: 0.3333333333333333


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in long_scalars
